# MDAnalysis Tutorial 

## Preparations

### Loading MDAnalysis

In [1]:
#!which python
import numpy as np
import MDAnalysis as mda
#from MDAnalysis.tests.datafiles import PSF, DCD, GRO, TRR
from MDAnalysis.analysis import rms
import pandas as pd
#import nglview as nv
import MDAnalysis.transformations
from MDAnalysis.analysis import diffusionmap, align, rms
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
mda.__version__

'2.6.1'

In [ ]:
!ls

## Basics

### Universe, AtomGroup and Trajectory

MDAnalysis is an object oriented library, meaning that data from your molecular simulations are made available through various MDAnalysis objects, which will be introduced here.

The first object in any analysis is the `Universe` object, which is the central data structure from which all other objects are created.  This is created through providing a topology (description of the system) and trajectory (description of the temporal data) file.

In [3]:
#In the case of our sim data, the XTC file is the trajectory_file and TPR is the topology_file, GRO can be used both as topology or trajectory depending on the information given
#In this case, I will not consider the GRO file and before I convert to pdb with trjconv, I will eliminate solvent and centre in CoM of system
#If you follow GROMACS tutorial instructions the XTC is md_0_1_noPBC.xtc
#PREFERRED ALTERNATIVE: gmx trjconv -s input.tpr -f input.xtc -o output.pdb --> mda.Universe('output.pdb')
#u = mda.Universe('topo.pdb', 'trj.pdb')
#u = mda.Universe(PSF, DCD)
u = mda.Universe('trajMolearn_stride_proteinH_center_unwrap.pdb', dt=1000)
#u= mda.Merge(protein)
print(u)

<Universe with 439 atoms>


In [4]:
aligner = align.AlignTraj(u, u, select='protein and not name H* and resid 47-59',
                          in_memory=True).run()

In [5]:
ag = u.select_atoms('all')
with MDAnalysis.Writer("alignToOneHelix.pdb", ag.n_atoms) as W:
    for ts in u.trajectory:
        W.write(ag)


/home/marmatt/miniforge3/envs/molearn/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/marmatt/miniforge3/envs/molearn/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1198: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."


In [6]:
# Alpha-synuclein

Rgyr2KKW = []
#Rmsd = []
#protein = u.select_atoms("protein and not name H*")
#print(len(protein))
for ts in u.trajectory:
    #R = mda.analysis.rms.RMSD(u, u, select='protein', ref_frame=0)
    #R = rms.rmsd(u, u)-->RIGHT COMMAND (although rmsd and not RMSD)
    #Rmsd.append((u.trajectory.time, R))
    #Rmsd.append((u.trajectory.time, protein.rmsd.RMSD()))
    Rgyr2KKW.append((u.trajectory.time, u.atoms.radius_of_gyration()))
Rgyr2KKW = np.array(Rgyr2KKW)
Rgyr2KKW[:,0] = Rgyr2KKW[:,0]/1000 
df_alpha_syn_rg = pd.DataFrame(Rgyr2KKW, columns=['Time (ns)', 'Rg'])
df_alpha_syn_rg.to_csv('alpha-syn_Rg_data.csv')

In [7]:
from MDAnalysis.analysis import diffusionmap, align, rms
#calculate pairwise-RMSD
#protein = u.select_atoms("protein and not name H*")
matrix = diffusionmap.DistanceMatrix(u, select='name CA').run()

df_alpha_syn_pairRMSD = pd.DataFrame(matrix.dist_matrix)

df_alpha_syn_pairRMSD.to_csv('alpha-syn_pairRMSD_data.csv')

/home/marmatt/miniforge3/envs/molearn/lib/python3.10/site-packages/MDAnalysis/analysis/diffusionmap.py:288: DeprecationWarning: The `dist_matrix` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.dist_matrix` instead.
  warnings.warn(wmsg, DeprecationWarning)


## Trajectory Analysis

The `Universe` binds together the static topology (which atoms, how are they connected, what un-changing properties do the atoms possess (such as partial charge), ...) and the changing coordinate information, which is stored in the trajectory.

The length of a trajectory (number of frames) is

In [ ]:
#to include timestep u = mda.Universe('topo.pdb', 'trj.pdb', dt=)
#u = mda.Universe('topo.pdb', 'trj.pdb')
#u = mda.Universe(PSF, DCD)
#WHY DO I NEED TO SPECIFY DT HERE, DT=10 Since i sample every 10ps and mdAnalysis defaults to 1ps RIGHT??
u = mda.Universe('traj_center_fit.pdb', dt=10)
#uw = mda.Universe('lyso.pdb', dt=10)
#ref = mda.Universe('mdAnalysis.pdb', dt=10)
len(u.trajectory)
#len(uw.trajectory)



In [ ]:
'''
Jasmin's universe function
'''
#def get_universe(file_path: str) -> tuple:
def get_universe() -> tuple:
    """
    get GRO, XTC files and return universe
    """
    #topology = file_path + "npt.gro"
    #trajectory = file_path + "md_0_1.xtc"
    topology = "npt.gro"
    trajectory = "md_0_1.xtc"
    u = mda.Universe(topology, trajectory, topology_format="GRO")
    return u 
#print(get_universe('~/Documents/asyn_setup_august_23/AnalysisOfFirstTrajectory/') )
print(get_universe())

In [ ]:
topology = "npt.gro"
trajectory = "md_0_1.xtc"
u = mda.Universe(topology, trajectory, topology_format="GRO")
len(u.trajectory)


In [ ]:
print(u)

The standard way to assess each time step (or frame) in a trajectory is to iterate over the `Universe.trajectory` attribute (which is an instance of Reader class):

In [ ]:
for ts in u.trajectory:
    print("Frame: %5d, Time: %8.3f ps" % (ts.frame, u.trajectory.time))
    print("Rgyr: %g A" % (u.atoms.radius_of_gyration(), ))
'''
for tz in uw.trajectory:
    print("Frame: %5d, Time: %8.3f ps" % (tz.frame, uw.trajectory.time))
    print("Rgyr: %g A" % (u.atoms.radius_of_gyration(), ))
'''

Rgyr: 35.3242 A
Frame:   142, Time: 1420.000 ps
Rgyr: 35.1764 A
Frame:   143, Time: 1430.000 ps
Rgyr: 35.4619 A
Frame:   144, Time: 1440.000 ps
Rgyr: 35.1966 A
Frame:   145, Time: 1450.000 ps
Rgyr: 35.3407 A
Frame:   146, Time: 1460.000 ps
Rgyr: 35.1544 A
Frame:   147, Time: 1470.000 ps
Rgyr: 35.1886 A
Frame:   148, Time: 1480.000 ps
Rgyr: 35.1157 A
Frame:   149, Time: 1490.000 ps
Rgyr: 35.4788 A
Frame:   150, Time: 1500.000 ps
Rgyr: 35.4505 A
Frame:   151, Time: 1510.000 ps
Rgyr: 35.3061 A
Frame:   152, Time: 1520.000 ps
Rgyr: 35.2369 A
Frame:   153, Time: 1530.000 ps
Rgyr: 35.0235 A
Frame:   154, Time: 1540.000 ps
Rgyr: 35.4291 A
Frame:   155, Time: 1550.000 ps
Rgyr: 34.9696 A
Frame:   156, Time: 1560.000 ps
Rgyr: 34.6533 A
Frame:   157, Time: 1570.000 ps
Rgyr: 34.8472 A
Frame:   158, Time: 1580.000 ps
Rgyr: 34.8591 A
Frame:   159, Time: 1590.000 ps
Rgyr: 34.6372 A
Frame:   160, Time: 1600.000 ps
Rgyr: 34.7391 A
Frame:   161, Time: 1610.000 ps
Rgyr: 34.8819 A
Frame:   162, Time: 1620

Rgyr: 35.1279 A
Frame:   340, Time: 3400.000 ps
Rgyr: 35.2452 A
Frame:   341, Time: 3410.000 ps
Rgyr: 35.3304 A
Frame:   342, Time: 3420.000 ps
Rgyr: 35.4425 A
Frame:   343, Time: 3430.000 ps
Rgyr: 35.2983 A
Frame:   344, Time: 3440.000 ps
Rgyr: 35.5247 A
Frame:   345, Time: 3450.000 ps
Rgyr: 35.0384 A
Frame:   346, Time: 3460.000 ps
Rgyr: 35.2297 A
Frame:   347, Time: 3470.000 ps
Rgyr: 34.9947 A
Frame:   348, Time: 3480.000 ps
Rgyr: 34.9225 A
Frame:   349, Time: 3490.000 ps
Rgyr: 34.3962 A
Frame:   350, Time: 3500.000 ps
Rgyr: 34.1774 A
Frame:   351, Time: 3510.000 ps
Rgyr: 34.2992 A
Frame:   352, Time: 3520.000 ps
Rgyr: 34.3289 A
Frame:   353, Time: 3530.000 ps
Rgyr: 34.1665 A
Frame:   354, Time: 3540.000 ps
Rgyr: 33.9417 A
Frame:   355, Time: 3550.000 ps
Rgyr: 33.8762 A
Frame:   356, Time: 3560.000 ps
Rgyr: 33.9155 A
Frame:   357, Time: 3570.000 ps
Rgyr: 33.9842 A
Frame:   358, Time: 3580.000 ps
Rgyr: 34.1545 A
Frame:   359, Time: 3590.000 ps
Rgyr: 34.1474 A
Frame:   360, Time: 3600

Rgyr: 32.2963 A
Frame:   526, Time: 5260.000 ps
Rgyr: 32.6342 A
Frame:   527, Time: 5270.000 ps
Rgyr: 32.582 A
Frame:   528, Time: 5280.000 ps
Rgyr: 32.4316 A
Frame:   529, Time: 5290.000 ps
Rgyr: 32.4985 A
Frame:   530, Time: 5300.000 ps
Rgyr: 32.2887 A
Frame:   531, Time: 5310.000 ps
Rgyr: 32.2689 A
Frame:   532, Time: 5320.000 ps
Rgyr: 32.0189 A
Frame:   533, Time: 5330.000 ps
Rgyr: 31.8641 A
Frame:   534, Time: 5340.000 ps
Rgyr: 31.9128 A
Frame:   535, Time: 5350.000 ps
Rgyr: 31.9711 A
Frame:   536, Time: 5360.000 ps
Rgyr: 31.8271 A
Frame:   537, Time: 5370.000 ps
Rgyr: 31.7886 A
Frame:   538, Time: 5380.000 ps
Rgyr: 31.9348 A
Frame:   539, Time: 5390.000 ps
Rgyr: 31.6933 A
Frame:   540, Time: 5400.000 ps
Rgyr: 31.8137 A
Frame:   541, Time: 5410.000 ps
Rgyr: 31.771 A
Frame:   542, Time: 5420.000 ps
Rgyr: 31.5161 A
Frame:   543, Time: 5430.000 ps
Rgyr: 31.2753 A
Frame:   544, Time: 5440.000 ps
Rgyr: 31.3745 A
Frame:   545, Time: 5450.000 ps
Rgyr: 31.2348 A
Frame:   546, Time: 5460.0

Rgyr: 36.352 A
Frame:   742, Time: 7420.000 ps
Rgyr: 36.0035 A
Frame:   743, Time: 7430.000 ps
Rgyr: 36.0442 A
Frame:   744, Time: 7440.000 ps
Rgyr: 36.0059 A
Frame:   745, Time: 7450.000 ps
Rgyr: 35.9594 A
Frame:   746, Time: 7460.000 ps
Rgyr: 35.9208 A
Frame:   747, Time: 7470.000 ps
Rgyr: 35.8502 A
Frame:   748, Time: 7480.000 ps
Rgyr: 35.9402 A
Frame:   749, Time: 7490.000 ps
Rgyr: 35.9363 A
Frame:   750, Time: 7500.000 ps
Rgyr: 35.6622 A
Frame:   751, Time: 7510.000 ps
Rgyr: 35.785 A
Frame:   752, Time: 7520.000 ps
Rgyr: 35.6529 A
Frame:   753, Time: 7530.000 ps
Rgyr: 35.5658 A
Frame:   754, Time: 7540.000 ps
Rgyr: 35.4783 A
Frame:   755, Time: 7550.000 ps
Rgyr: 35.6411 A
Frame:   756, Time: 7560.000 ps
Rgyr: 35.916 A
Frame:   757, Time: 7570.000 ps
Rgyr: 35.902 A
Frame:   758, Time: 7580.000 ps
Rgyr: 35.6122 A
Frame:   759, Time: 7590.000 ps
Rgyr: 35.4223 A
Frame:   760, Time: 7600.000 ps
Rgyr: 35.1599 A
Frame:   761, Time: 7610.000 ps
Rgyr: 35.7254 A
Frame:   762, Time: 7620.000

Rgyr: 33.7582 A
Frame:   927, Time: 9270.000 ps
Rgyr: 33.7752 A
Frame:   928, Time: 9280.000 ps
Rgyr: 33.5877 A
Frame:   929, Time: 9290.000 ps
Rgyr: 33.1451 A
Frame:   930, Time: 9300.000 ps
Rgyr: 32.8423 A
Frame:   931, Time: 9310.000 ps
Rgyr: 33.2355 A
Frame:   932, Time: 9320.000 ps
Rgyr: 33.3565 A
Frame:   933, Time: 9330.000 ps
Rgyr: 33.6364 A
Frame:   934, Time: 9340.000 ps
Rgyr: 33.3475 A
Frame:   935, Time: 9350.000 ps
Rgyr: 33.4651 A
Frame:   936, Time: 9360.000 ps
Rgyr: 33.3455 A
Frame:   937, Time: 9370.000 ps
Rgyr: 33.372 A
Frame:   938, Time: 9380.000 ps
Rgyr: 33.416 A
Frame:   939, Time: 9390.000 ps
Rgyr: 33.6895 A
Frame:   940, Time: 9400.000 ps
Rgyr: 33.9064 A
Frame:   941, Time: 9410.000 ps
Rgyr: 33.8393 A
Frame:   942, Time: 9420.000 ps
Rgyr: 34.1861 A
Frame:   943, Time: 9430.000 ps
Rgyr: 34.2157 A
Frame:   944, Time: 9440.000 ps
Rgyr: 34.0603 A
Frame:   945, Time: 9450.000 ps
Rgyr: 34.0126 A
Frame:   946, Time: 9460.000 ps
Rgyr: 34.4427 A
Frame:   947, Time: 9470.0

...

Rgyr: 29.6695 A
Frame:  4109, Time: 41090.000 ps
Rgyr: 29.722 A
Frame:  4110, Time: 41100.000 ps
Rgyr: 29.7093 A
Frame:  4111, Time: 41110.000 ps
Rgyr: 29.5252 A
Frame:  4112, Time: 41120.000 ps
Rgyr: 29.4707 A
Frame:  4113, Time: 41130.000 ps
Rgyr: 29.2374 A
Frame:  4114, Time: 41140.000 ps
Rgyr: 29.3438 A
Frame:  4115, Time: 41150.000 ps
Rgyr: 29.6202 A
Frame:  4116, Time: 41160.000 ps
Rgyr: 29.7085 A
Frame:  4117, Time: 41170.000 ps
Rgyr: 29.7311 A
Frame:  4118, Time: 41180.000 ps
Rgyr: 29.7556 A
Frame:  4119, Time: 41190.000 ps
Rgyr: 29.8906 A
Frame:  4120, Time: 41200.000 ps
Rgyr: 29.7061 A
Frame:  4121, Time: 41210.000 ps
Rgyr: 29.8107 A
Frame:  4122, Time: 41220.000 ps
Rgyr: 29.6206 A
Frame:  4123, Time: 41230.000 ps
Rgyr: 29.6019 A
Frame:  4124, Time: 41240.000 ps
Rgyr: 29.6789 A
Frame:  4125, Time: 41250.000 ps
Rgyr: 29.8396 A
Frame:  4126, Time: 41260.000 ps
Rgyr: 29.7097 A
Frame:  4127, Time: 41270.000 ps
Rgyr: 29.7242 A
Frame:  4128, Time: 41280.000 ps
Rgyr: 29.626 A
Frame:

Rgyr: 28.2836 A
Frame:  4326, Time: 43260.000 ps
Rgyr: 28.4118 A
Frame:  4327, Time: 43270.000 ps
Rgyr: 28.7344 A
Frame:  4328, Time: 43280.000 ps
Rgyr: 28.8336 A
Frame:  4329, Time: 43290.000 ps
Rgyr: 28.821 A
Frame:  4330, Time: 43300.000 ps
Rgyr: 28.5951 A
Frame:  4331, Time: 43310.000 ps
Rgyr: 28.6146 A
Frame:  4332, Time: 43320.000 ps
Rgyr: 28.4719 A
Frame:  4333, Time: 43330.000 ps
Rgyr: 28.6237 A
Frame:  4334, Time: 43340.000 ps
Rgyr: 28.5559 A
Frame:  4335, Time: 43350.000 ps
Rgyr: 28.6001 A
Frame:  4336, Time: 43360.000 ps
Rgyr: 28.5622 A
Frame:  4337, Time: 43370.000 ps
Rgyr: 28.4648 A
Frame:  4338, Time: 43380.000 ps
Rgyr: 28.3484 A
Frame:  4339, Time: 43390.000 ps
Rgyr: 28.2007 A
Frame:  4340, Time: 43400.000 ps
Rgyr: 28.2229 A
Frame:  4341, Time: 43410.000 ps
Rgyr: 28.1902 A
Frame:  4342, Time: 43420.000 ps
Rgyr: 28.1971 A
Frame:  4343, Time: 43430.000 ps
Rgyr: 28.4459 A
Frame:  4344, Time: 43440.000 ps
Rgyr: 28.4341 A
Frame:  4345, Time: 43450.000 ps
Rgyr: 28.4156 A
Frame

Frame:  4513, Time: 45130.000 ps
Rgyr: 28.8406 A
Frame:  4514, Time: 45140.000 ps
Rgyr: 28.5939 A
Frame:  4515, Time: 45150.000 ps
Rgyr: 28.3645 A
Frame:  4516, Time: 45160.000 ps
Rgyr: 28.4376 A
Frame:  4517, Time: 45170.000 ps
Rgyr: 28.6243 A
Frame:  4518, Time: 45180.000 ps
Rgyr: 28.6963 A
Frame:  4519, Time: 45190.000 ps
Rgyr: 28.4347 A
Frame:  4520, Time: 45200.000 ps
Rgyr: 28.5859 A
Frame:  4521, Time: 45210.000 ps
Rgyr: 28.4168 A
Frame:  4522, Time: 45220.000 ps
Rgyr: 28.1606 A
Frame:  4523, Time: 45230.000 ps
Rgyr: 28.1809 A
Frame:  4524, Time: 45240.000 ps
Rgyr: 28.6588 A
Frame:  4525, Time: 45250.000 ps
Rgyr: 28.5468 A
Frame:  4526, Time: 45260.000 ps
Rgyr: 28.7806 A
Frame:  4527, Time: 45270.000 ps
Rgyr: 29.0515 A
Frame:  4528, Time: 45280.000 ps
Rgyr: 28.5798 A
Frame:  4529, Time: 45290.000 ps
Rgyr: 28.9193 A
Frame:  4530, Time: 45300.000 ps
Rgyr: 28.5635 A
Frame:  4531, Time: 45310.000 ps
Rgyr: 28.865 A
Frame:  4532, Time: 45320.000 ps
Rgyr: 28.7587 A
Frame:  4533, Time: 4

Rgyr: 28.5611 A
Frame:  4708, Time: 47080.000 ps
Rgyr: 28.6018 A
Frame:  4709, Time: 47090.000 ps
Rgyr: 28.5595 A
Frame:  4710, Time: 47100.000 ps
Rgyr: 28.5689 A
Frame:  4711, Time: 47110.000 ps
Rgyr: 28.7412 A
Frame:  4712, Time: 47120.000 ps
Rgyr: 28.3906 A
Frame:  4713, Time: 47130.000 ps
Rgyr: 28.4638 A
Frame:  4714, Time: 47140.000 ps
Rgyr: 28.5271 A
Frame:  4715, Time: 47150.000 ps
Rgyr: 28.3589 A
Frame:  4716, Time: 47160.000 ps
Rgyr: 28.3002 A
Frame:  4717, Time: 47170.000 ps
Rgyr: 28.1088 A
Frame:  4718, Time: 47180.000 ps
Rgyr: 28.2956 A
Frame:  4719, Time: 47190.000 ps
Rgyr: 28.3149 A
Frame:  4720, Time: 47200.000 ps
Rgyr: 28.4088 A
Frame:  4721, Time: 47210.000 ps
Rgyr: 28.3532 A
Frame:  4722, Time: 47220.000 ps
Rgyr: 28.4869 A
Frame:  4723, Time: 47230.000 ps
Rgyr: 28.3923 A
Frame:  4724, Time: 47240.000 ps
Rgyr: 28.6508 A
Frame:  4725, Time: 47250.000 ps
Rgyr: 28.2958 A
Frame:  4726, Time: 47260.000 ps
Rgyr: 28.4526 A
Frame:  4727, Time: 47270.000 ps
Rgyr: 28.6045 A
Fram

Rgyr: 28.6234 A
Frame:  4918, Time: 49180.000 ps
Rgyr: 28.3804 A
Frame:  4919, Time: 49190.000 ps
Rgyr: 28.4792 A
Frame:  4920, Time: 49200.000 ps
Rgyr: 28.4142 A
Frame:  4921, Time: 49210.000 ps
Rgyr: 28.4526 A
Frame:  4922, Time: 49220.000 ps
Rgyr: 28.6175 A
Frame:  4923, Time: 49230.000 ps
Rgyr: 28.6469 A
Frame:  4924, Time: 49240.000 ps
Rgyr: 28.6873 A
Frame:  4925, Time: 49250.000 ps
Rgyr: 28.9731 A
Frame:  4926, Time: 49260.000 ps
Rgyr: 28.8987 A
Frame:  4927, Time: 49270.000 ps
Rgyr: 28.9638 A
Frame:  4928, Time: 49280.000 ps
Rgyr: 28.8698 A
Frame:  4929, Time: 49290.000 ps
Rgyr: 28.8037 A
Frame:  4930, Time: 49300.000 ps
Rgyr: 29.0708 A
Frame:  4931, Time: 49310.000 ps
Rgyr: 28.8311 A
Frame:  4932, Time: 49320.000 ps
Rgyr: 28.6727 A
Frame:  4933, Time: 49330.000 ps
Rgyr: 28.4043 A
Frame:  4934, Time: 49340.000 ps
Rgyr: 28.697 A
Frame:  4935, Time: 49350.000 ps
Rgyr: 28.6012 A
Frame:  4936, Time: 49360.000 ps
Rgyr: 28.717 A
Frame:  4937, Time: 49370.000 ps
Rgyr: 28.4516 A
Frame:

The time attribute contains the current time step. The Reader only contains information about one time step: imagine a cursor or pointer moving along the trajectory file. Where the cursor points, there’s you current coordinates, frame number, and time.

Normally you will collect the data in a list or array, e.g.

In [ ]:
# Alpha-synuclein

Rgyr2KKW = []
#Rmsd = []
#protein = u.select_atoms("protein and not name H*")
#print(len(protein))
for ts in u.trajectory:
    #R = mda.analysis.rms.RMSD(u, u, select='protein', ref_frame=0)
    #R = rms.rmsd(u, u)-->RIGHT COMMAND (although rmsd and not RMSD)
    #Rmsd.append((u.trajectory.time, R))
    #Rmsd.append((u.trajectory.time, protein.rmsd.RMSD()))
    Rgyr2KKW.append((u.trajectory.time, protein.radius_of_gyration()))
Rgyr2KKW = np.array(Rgyr2KKW)
Rgyr2KKW[:,0] = Rgyr2KKW[:,0]/1000 
df_alpha_syn_rg = pd.DataFrame(Rgyr2KKW, columns=['Time (ns)', 'Rg'])
df_alpha_syn_rg.to_csv('alpha-syn_Rg_data.csv')

#print(alpha_syn_pairRMSD)
#Rmsd = np.array(Rmsd)

#R2KKW = rms.RMSD(u, u, select='backbone', ref_frame=0)
#R2KKW.run()
#R2KKW.results.rmsd.shape

# Lysosyme
'''
RgyrLyso = []
#Rmsd = []
protein = uw.select_atoms("protein")
for ts in uw.trajectory:
    #R = mda.analysis.rms.RMSD(u, u, select='protein', ref_frame=0)
    #R = rms.rmsd(u, u)-->RIGHT COMMAND (although rmsd and not RMSD)
    #Rmsd.append((u.trajectory.time, R))
    #Rmsd.append((u.trajectory.time, protein.rmsd.RMSD()))
    RgyrLyso.append((uw.trajectory.time, protein.radius_of_gyration()))
RgyrLyso = np.array(RgyrLyso)
#Rmsd = np.array(Rmsd)0
RLyso = rms.RMSD(uw, uw, select='backbone', ref_frame=0)
RLyso.run()
RLyso.results.rmsd.shape
'''

In [ ]:
from MDAnalysis.analysis import diffusionmap, align, rms
#calculate pairwise-RMSD
protein = u.select_atoms("protein and not name H*")
matrix = diffusionmap.DistanceMatrix(protein, select='name CA').run()

df_alpha_syn_pairRMSD = pd.DataFrame(matrix.dist_matrix)

df_alpha_syn_pairRMSD.to_csv('alpha-syn_pairRMSD_data.csv')

In [ ]:
# RMSF calcualtion
# Alpha-synuclein
from MDAnalysis.analysis import rms, align

#average = align.AverageStructure(u, u, select='protein and name CA',ref_frame=0).run()
#ref = average.results.universe


#aligner = align.AlignTraj(u, ref,select='protein and name CA',in_memory=True).run()
#print('aligning done')
c_alphas_alpha_syn = u.select_atoms('protein and name CA')
RMSF_alpha_syn = rms.RMSF(c_alphas_alpha_syn).run()
df_alpha_syn_rmsf = pd.DataFrame(RMSF_alpha_syn, columns=['RMSF'])
print('alpha syn done')



In [ ]:
# Lysosyme
average_uw = align.AverageStructure(uw, uw, select='protein and name CA',
                                 ref_frame=0).run()
ref_uw = average_uw.results.universe

aligner = align.AlignTraj(uw, ref_uw,
                          select='protein and name CA',
                          in_memory=True).run()
# alignment done

c_alphas_lys = uw.select_atoms('protein and name CA')
RMSF_lys = rms.RMSF(c_alphas_lys).run()


In [ ]:
# Adjusting to nanosectonds
# alpha-syn results rmsd
R2KKW.results.rmsd[:,1] = R2KKW.results.rmsd[:,1]/1000
'''
# Lysosyme results rmsd
RLyso.results.rmsd[:,1] = RLyso.results.rmsd[:,1]/1000
'''

In [ ]:
# RMSD plot
df_alpha_syn = pd.DataFrame(R2KKW.results.rmsd, columns=['Frame', 'Time (ns)', 'RMSD'])
#dw_lysozyme = pd.DataFrame(RLyso.results.rmsd, columns=['Frame', 'Time (ns)', 'RMSD'])

In [ ]:
# Adjusting to nanoseconds
Rgyr2KKW[:,0] = Rgyr2KKW[:,0]/1000 
#RgyrLyso[:,0] = RgyrLyso[:,0]/1000

In [ ]:
# Rg
alpha_syn_rg = pd.DataFrame(Rgyr2KKW, columns=['Time (ns)', 'Rg'])
#lysozyme_rg = pd.DataFrame(RgyrLyso, columns=['Time (ns)', 'Rg'])

In [ ]:
df_alpha_syn['Rg'] = alpha_syn_rg['Rg']
#dw_lysozyme['Rg'] = lysozyme_rg['Rg']

In [ ]:
# Saving data
df_alpha_syn.to_csv('alpha-syn_data.csv')
#dw_lysozyme.to_csv('lysozyme_data.csv')

In [ ]:
# RMSF results

print(len(RMSF_alpha_syn.rmsf))
#print(len(RMSF_lys.rmsf))
np.savetxt('alpha_syn_rmsf.csv', RMSF_alpha_syn.rmsf)
#np.savetxt('lysosyme_rmsf.csv', RMSF_lys.rmsf)

### Now let's plot everything

In [ ]:
##R2KKWCrystal =
plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True

#THE FOLLOWING SHOULD BE RIGHT BUT WHY DOES IT NOT READ THE TIMESTEP?
df = pd.DataFrame(R2KKW.results.rmsd, columns=['Frame', 'Time (ns)', 'Alpha-synuclein'])
#dw = pd.DataFrame(RLyso.results.rmsd, columns=['Frame', 'Time (ns)', 'Lysozyme'])
print(df)


ax = df.plot(x='Time (ns)', y=['Alpha-synuclein'], kind='line')
#dw.plot(ax=ax, x='Time (ps)', y=['Lysozyme'], kind='line')
plt.title('RMSD over time')
ax.set_ylabel(r'RMSD($\AA$)')
plt.show()


#plt.plot(R2KKW.results.rmsd)
#plt.plot(RLyso.results.rmsd)
'''

'''

In [ ]:
df = pd.DataFrame(Rgyr2KKW, columns=['Time (ps)', 'Alpha-synuclein'])
dw = pd.DataFrame(RgyrLyso, columns=['Time (ps)', 'Lysozyme'])
print(df)
print(dw)
# quick plot
#import matplotlib.pyplot as plt
#ax = plt.subplot(111)
#ax.plot(Rgyr[:,0], Rgyr[:,1], 'r--', lw=2, label=r"$R_G$")
ax = df.plot(x='Time (ps)', y=['Alpha-synuclein'], kind='line')
dw.plot(ax=ax, x='Time (ps)', y=['Lysozyme'], kind='line')
ax.set_xlabel("time (ps)")
ax.set_ylabel(r"radius of gyration $R_G$ ($\AA$)")
ax.figure.savefig("Rgyr.pdf")
plt.draw()
plt.title('Radius of gyration over time')

## Writing Coordinates

MDAnalysis also supports writing of data in a range of file formats (see the [Table of Supported Coordinate Formats](https://pythonhosted.org/MDAnalysis/documentation_pages/coordinates/init.html#id1) for details). MDAnalysis supports both single frame writers (such as a simple PDB or GRO file) and trajectory writers (e.g. XTC, DCD, but also multi-frame PDB files).

### Single frames

The most straightforward way to write to a file that can only hold a single frame is to use the write() method of any AtomGroup as already also shown under Processing AtomGroups. For instance, to only write out the protein without solvent to a file in GRO format:

In [ ]:
from MDAnalysis.tests.datafiles import PDB
uPDB = MDAnalysis.Universe(PDB)
protein = uPDB.select_atoms("protein")
protein.write("protein.gro")

MDAnalysis uses the file suffix to determine the output file format (unless the format keyword is specified) and will raise an exception if it is not suitable for single frame writing.

In [ ]:
del uPDB  # clean up this example
os.unlink("protein.gro")

### Trajectories

The typical use pattern is to

1. Get a trajectory writer with `MDAnalysis.Writer()` (which is the same as `MDAnalysis.coordinates.core.writer()`), typically specifying in advance how many atoms a frame will contain.
2. Use the `write()` method to write a new time step to the trajectory.
3. Close the trajectory with `close()` (although it is recommended to simply use the writer with the `with` statement and have the context manager close the file automatically).

#### Example: Protein-only trajectory

In practice, the second step is typically repeated in a loop as in the example below:

In [ ]:
import MDAnalysis
from MDAnalysis.tests.datafiles import PDB, XTC

uXTC = MDAnalysis.Universe(PDB, XTC)
protein = uXTC.select_atoms("protein")
with MDAnalysis.Writer("protein.xtc", protein.n_atoms) as W:
    for ts in u.trajectory:
        W.write(protein)

The loop steps through the input trajectory frame by frame. The coordinates of the selection (the AtomGroup protein) change accordingly and are then written as a new frame into the output trajectory.

The output trajectory only contains the coordinates of the protein. For this trajectory to be useful, a protein-only topology file also has to be stored, as in the example under Single frames.

In [ ]:
del uXTC  # clean up this example
os.unlink("protein.xtc")

#### Example: Saving dynamic per-atom properties in B-factor

It is often very useful to project per-atom properties on the structure. A common approach is to save scalar values in the B-factor field of a PDB file and then color atoms by B-factor (also known as temperature factor or just “beta”).

The following example computes the shift of each atom in AdK relative to a reference structure (line 23). We take as reference the closed conformation (after a structural superposition on the CORE domain with alignto()). The shifts are written into the B-factor with the set_bfactor() method of AtomGroup. Each frame is written out as part of a multi-frame PDB file:

In [ ]:
# project a dynamic property on the structure using the B-factor field

import numpy as np
import MDAnalysis
import MDAnalysis.analysis.align
from MDAnalysis.core.topologyattrs import Bfactors

from MDAnalysis.tests.datafiles import PSF, DCD


u = MDAnalysis.Universe(PSF, DCD)
u.add_TopologyAttr(Bfactors(np.zeros(len(u.atoms))))
ref = MDAnalysis.Universe(PSF, DCD)  # copy of u

CORE_selection = "resid 1:29 or resid 60:121 or resid 160:214"
pdbtrj = "adk_distance_bfac.pdb"

with MDAnalysis.Writer(pdbtrj, multiframe=True, n_atoms=u.atoms.n_atoms) as PDB:
    # reference coordinates: set to first frame
    ref.trajectory[0]
    # iterate through our trajectory
    for ts in u.trajectory:
        # superimpose on the reference CORE (at t=0)
        rmsd = MDAnalysis.analysis.align.alignto(u.atoms, ref.atoms, select=CORE_selection)
        distances = np.sqrt(np.sum((u.atoms.positions - ref.atoms.positions)**2, axis=1))
        # project displacement on structure via bfactor field
        u.atoms.bfactors = distances
        PDB.write(u.atoms)
        print("Frame {0}: CORE RMSD before/after superposition: {1[0]:.1f} / {1[1]:.1f} A. "
              "min-max displacement: {2:.1f}...{3:.1f} A".format(ts.frame, rmsd, distances.min(), distances.max()))

print("Wrote PDB trajectory {0} with distances in bfactor field".format(pdbtrj))

## Using the MDAnalysis.analysis Modules

MDAnalysis comes with a number of existing analysis code in the `MDAnalysis.analysis` module and example scripts (see also the Examples on the MDAnalysis wiki).

### RMSD

As an example we will use the MDAnalysis.analysis.rms.rmsd() function from the MDAnalysis.analysis.rms module. It computes the coordinate root mean square distance between two sets of coordinates. For example for the AdK trajectory the backbone RMSD between first and last frame is

In [ ]:
import MDAnalysis.analysis.rms
u = MDAnalysis.Universe(PSF, DCD)
bb = u.select_atoms('backbone')
A = bb.positions  # coordinates of first frame
u.trajectory[-1]      # forward to last frame
B = bb.positions  # coordinates of last frame
MDAnalysis.analysis.rms.rmsd(A,B)

### Superposition of structure

In order to superimpose two structures in a way that minimizes the RMSD we have functions in the MDAnalysis.analysis.align module.

The example uses files provided as part of the MDAnalysis test suite (in the variables PSF, DCD, and PDB_small). For all further examples execute first

In [ ]:
import MDAnalysis
from MDAnalysis.analysis import align, rms
from MDAnalysis.tests.datafiles import PSF, DCD, PDB_small

In the simplest case, we can simply calculate the C-alpha RMSD between two structures, using rmsd():

In [ ]:
ref = MDAnalysis.Universe(PDB_small)
mobile = MDAnalysis.Universe(PSF,DCD)
rms.rmsd(mobile.atoms.CA.positions, ref.atoms.CA.positions)

Note that in this example translations have not been removed. In order to look at the pure rotation one needs to superimpose the centres of mass (or geometry) first:

In [ ]:
ref0 =  ref.atoms.CA.positions - ref.atoms.CA.center_of_mass()
mobile0 =  mobile.atoms.CA.positions - mobile.atoms.CA.center_of_mass()
rms.rmsd(mobile0, ref0)

The rotation matrix that superimposes mobile on ref while minimizing the CA-RMSD is obtained with the rotation_matrix() function

In [ ]:
R, rmsd = align.rotation_matrix(mobile0, ref0)
print(rmsd)

In [ ]:
print(R)

Putting all this together one can superimpose all of mobile onto ref:

In [ ]:
mobile.atoms.translate(-mobile.atoms.CA.center_of_mass())
mobile.atoms.rotate(R)
mobile.atoms.translate(ref.atoms.CA.center_of_mass())
mobile.atoms.write("mobile_on_ref.pdb")

#### Exercise 5

Use the above in order to investigate how rigid the CORE, NMP, and LID domains are during the transition: Compute time series of the CA RMSD of each domain relative to its own starting structure, when superimposed on the starting structure.

The code contains a function superpose() and rmsd(). The latter is marginally faster because we only need the calculated RMSD and not the full rotation matrix. (We are calling the lower-level function MDAnalysis.core.qcprot.CalcRMSDRotationalMatrix() directly, which has somewhat non-intuitive calling conventions). superpose() also does the superposition of the mobile group to the references (but alignto() is actually a more flexible tool for doing this). Otherwise it is mostly book-keeping, which is solved by organizing everything in dictionaries with keys “CORE”, “NMP”, “LID”.

In [ ]:
import numpy as np
from MDAnalysis.analysis.align import rotation_matrix
from MDAnalysis.lib.qcprot import CalcRMSDRotationalMatrix

def superpose(mobile, xref0, xref_com=None):
    """Superpose the AtomGroup *mobile* onto the coordinates *xref0* centered at the orgin.

    The original center of mass of the reference group *xref_com* must
    be supplied or the superposition is done at the origin of the
    coordinate system.
    """
    # 995 us
    xref_com = xref_com if xref_com is not None else np.array([0., 0., 0.])
    xmobile0 = mobile.positions - mobile.center_of_mass()
    R, rmsd = rotation_matrix(xmobile0, xref0)
    mobile.rotate(R)
    mobile.translate(xref_com)
    return rmsd

def rmsd(mobile, xref0):
    """Calculate optimal RMSD for AtomGroup *mobile* onto the coordinates *xref0* centered at the orgin.

    The coordinates are not changed. No mass weighting.
    """
    # 738 us
    xmobile0 = mobile.positions - mobile.center_of_mass()
    return CalcRMSDRotationalMatrix(xref0.T.astype(np.float64), xmobile0.T.astype(np.float64), mobile.n_atoms, None, None)


if __name__ == "__main__":
    import MDAnalysis
    import matplotlib
    import matplotlib.pyplot as plt

    # load AdK DIMS trajectory
    from MDAnalysis.tests.datafiles import PSF, DCD
    u = MDAnalysis.Universe(PSF, DCD)

    # one AtomGroup per domain
    domains = {
        'CORE': u.select_atoms("(resid 1:29 or resid 60:121 or resid 160:214) and name CA"),
        'LID': u.select_atoms("resid 122-159 and name CA"),
        'NMP': u.select_atoms("resid 30-59 and name CA"),
        }
    colors = {'CORE': 'black', 'NMP': 'blue', 'LID': 'red'}

    u.trajectory[0]   # rewind trajectory
    xref0 = dict((name, g.positions - g.center_of_mass()) for name,g in domains.iteritems())

    nframes = len(u.trajectory)
    results = dict((name, np.zeros((nframes, 2), dtype=np.float64)) for name in domains)

    for iframe,ts in enumerate(u.trajectory):
        for name, g in domains.iteritems():
            results[name][iframe, :] = u.trajectory.time, rmsd(g, xref0[name])


    # plot
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    for name in "CORE", "NMP", "LID":
        data = results[name]
        ax.plot(data[:,0], data[:,1], linestyle="-", color=colors[name], lw=2, label=name)
    ax.legend(loc="best")
    ax.set_xlabel(r"time  $t$ (ps)")
    ax.set_ylabel(r"C$_\alpha$ RMSD from $t=0$, $\rho_{\mathrm{C}_\alpha}$ ($\AA$)")

    for ext in ('svg', 'pdf', 'png'):
        fig.savefig("AdK_domain_rigidity.{0}".format(ext))